In [ ]:
with open('input.txt') as file:
    data = file.read()
    
data = data.split('\n')

In [ ]:
# Silver
def read_map(data):
    # Function to parse map data into 2D array
    map_data = []
    for ii,row in enumerate(data):

        # Check starting symbol
        if '^' in row:
            pos = [ii,row.index('^')]
            row = row.replace('^','.')
        
        map_data_row = [0] * len(row) # Initialise empty row
        for jj, coordinate in enumerate(row):
            if coordinate == '.':
                map_data_row[jj] = 0
            elif coordinate == '#':
                map_data_row[jj] = 1
        map_data.append(map_data_row)
    return pos, map_data

class Map():

    def __init__(self,pos,map_data):
        # Read initial map
        self.pos = pos
        self.map_data = map_data
        self.shape = [len(map_data), len(map_data[0])]
        self.patrol_path = [[0 for col in range(self.shape[1])] for row in range(self.shape[0])]
        self.speed_list = [[-1, 0], [0, 1], [1, 0], [0, -1]] # List to help keep track on directions

        self.speed = [-1, 0]
        self.speed_ptr = 0
        

    def step_forward(self):
        
        next_pos = [self.pos[0] + self.speed[0], self.pos[1] + self.speed[1]]
        if next_pos[0] >= self.shape[0] or next_pos[0] < 0 or next_pos[1] >= self.shape[1] or next_pos[1] < 0 :
            self.patrol_path[self.pos[0]][self.pos[1]] = 1
            return False
        

        if self.map_data[next_pos[0]][next_pos[1]]:

            # Update speed
            self.speed_ptr = (self.speed_ptr + 1) % 4
            self.speed = self.speed_list[self.speed_ptr]

        else:

            self.patrol_path[self.pos[0]][self.pos[1]] = 1
            self.pos = next_pos
            
        
        return True

        
pos, map_data = read_map(data)
patrol_map = Map(pos, map_data)

is_valid = True
while is_valid:
    is_valid = patrol_map.step_forward()

sum([sum(row) for row in patrol_map.patrol_path])

In [ ]:
from copy import deepcopy

# Gold, little brute force. about 10 sec to run
class Map():

    def __init__(self,pos, map_data):

        # Read initial map
        self.map_data = map_data
        self.pos = pos # Current position
        self.speed_list = [[-1, 0], [0, 1], [1, 0], [0, -1]] # List to help keep track on directions
        self.shape = [len(map_data),len(map_data[0])]
        self.positions = [] # Previous positions
        self.looping = False

        self.speed = [-1, 0]
        self.speed_ptr = 0

        # Patrol path 4 x 130 x 130. If we hit same place with same direction we know that we are looping
        self.patrol_path = [[[0 for col in range(self.shape[1])] for row in range(self.shape[0])] for speed in range(4)]

    def step_forward(self):
        # Calculate next position
        next_pos = [self.pos[0] + self.speed[0], self.pos[1] + self.speed[1]]
        
        # Check OB
        if next_pos[0] >= self.shape[0] or next_pos[0] < 0 or next_pos[1] >= self.shape[1] or next_pos[1] < 0:        
            return False
        
        # Check looping
        if self.patrol_path[self.speed_ptr][next_pos[0]][next_pos[1]]:
            self.looping = True
            return False
        
        self.patrol_path[self.speed_ptr][self.pos[0]][self.pos[1]] = 1
        
        # On obstacle hit
        if self.map_data[next_pos[0]][next_pos[1]]:
            self.speed_ptr = (self.speed_ptr + 1) % 4
            self.speed = self.speed_list[self.speed_ptr]
        # Otherwise
        else:
            self.pos = next_pos
            self.positions.append((next_pos[0], next_pos[1]))
            
        
        return True
        

pos, map_data = read_map(data)

# Single pass throught with the original map
patrol_map = Map(pos, map_data)
is_valid = True
while is_valid:
    is_valid = patrol_map.step_forward()



# Backtrack and change one coordinate at the time
# Use set to reduce overlapping pixels.
positions = set(patrol_map.positions)
s = 0
for ii,position in enumerate(positions):

    # Temporaly copy the map
    map_data_temp = deepcopy(map_data)

    # Add obstacle
    map_data_temp[position[0]][position[1]] = 1

    # Create map object
    patrol_map = Map(pos,map_data_temp) #

    # Pass throught map
    is_valid = True
    while is_valid:
        is_valid = patrol_map.step_forward()
    # Check whether map was looping or OB
    if patrol_map.looping:
        s += 1

    if ii % 100 == 0:
        print(f'Iteration {ii} / {len(set(positions))}')

    
    

s